In [231]:
#Função para extrair dados em formato CSV
def Extrair_Dados_Gerais(caminho=str,delimitador=';',pular_linha=None):
    '''Se trata de uma função para extrair dados de uma rede elétrica em CSV
    \n A função funciona com o uso de duas variáveis nos seguintes moldes:
    \n caminho=str
    \n delimitador=';'
    \n Retorna a média, desvio padrão, mediana, máximo, mínimo e data.
    '''
    
    import numpy as np
    import pandas as pd

    #Tentar ler o arquivo CSV no padrão usado pela empresa
    if pular_linha==None:
        dados=pd.read_csv(caminho,delimiter=delimitador,skiprows=pular_linha)
        #Renomeando coluna
        dados.columns=dados.iloc[1]
        #Excluindo linhas de texto
        dados.drop(dados.index[:2], inplace=True)
    #Se nessecitar pular linha, usar modelo diferente
    else:
        dados=pd.read_csv(caminho,skiprows=pular_linha)
        #Renomeando coluna
        dados.columns=dados.iloc[0]
        #Excluindo linhas de texto
        dados.drop(dados.index[:1], inplace=True)
    #Resetando Indice
    dados.reset_index(inplace=True,drop=True)
    #Salvando data
    data=dados['Date'][1]
    #Excluindo colunas não númericas
    dados.drop(columns=['Date','Time','PDmP','PDmIA_D/T','PDmIB_D/T','PDmIC_D/T','PDmIAVG_D/T','PDmP_D/T','PDmQ_D/T','PDmS_D/T'],inplace=True)
    #Retirando dados entre pontos ABC e mantendo somente o consolidado
    #Sugiro esse processo para facilitar visualização,
    #dado o grande número de variáveis
    dados.drop(columns=['UA','UB','UC','UN','UTHDA','UTHDB','UTHDC','IA','IB','IC','ITHDA','ITHDB','ITHDC','FA','FB','FC','PFA','PFB','PFC','PA','PB','PC'],inplace=True)
    dados.drop(columns=['QA','QB','QC','SA','SB','SC','EPA','EPB','EPC','EQA','EQB','EQC','ESA','ESB','ESC','DmIA','DmIB','DmIC','PDmIA','PDmIB','PDmIC'],inplace=True)
    #Convertendo texto para número
    dados=dados.apply(pd.to_numeric)
    #Declarando variável
    consolidado=pd.DataFrame()
    #Calculando variáveis
    consolidado['Média']=pd.DataFrame(round(dados.mean(),1))
    consolidado['Desvio padrão']=pd.DataFrame(round(dados.std(),1))
    consolidado['Mediana']=pd.DataFrame(round(dados.median(),1))
    consolidado['Máximo']=pd.DataFrame(round(dados.max(),1))
    consolidado['Mínimo']=pd.DataFrame(round(dados.min(),1))
    #Trazendo Data para a tabela
    consolidado['Data']=[data]*len(consolidado)
   
    return consolidado

In [232]:
#Função para formar relatório geral completo
def Relatorio_Geral(*args):
    '''Se trata de uma função para unir dataframes
    \n A função funciona com a entrada de n dataframes extraidos
    '''
    import pandas as pd

    #Concatenando data frames
    Relatorio=pd.concat(args)
    #Recriando indice para melhor organização
    Relatorio['Variáveis']=Relatorio.index
    Relatorio.reset_index(inplace=True,drop=True)
    Relatorio=Relatorio.set_index(['Data','Variáveis'])
    return Relatorio

In [233]:
#Função para encontrar as 5 maiores harmonicas do dia
def Maiores_harmonicas(caminhos=list,pular_linha=1):
    '''Função que recebe qualquer planilha de harmonicas (Tensão e corrente). 
    \nRetorna as 10 maiores ocorrencias
    '''
    import pandas as pd
    import numpy as np
    
    #Criação de lista para guardar resultados
    save=list()
    #Declarando data frame final
    Relatorio=pd.DataFrame()
    for endereço in caminhos:
        dados=pd.read_csv(endereço,skiprows=pular_linha)
        #Renomeando colunas
        dados.columns=dados.iloc[0]
        #Excluindo linhas de texto
        dados.drop(columns=['Time'],inplace=True)
        dados.drop(dados.index[:1], inplace=True)
        #Agrupando variáveis
        consolidado=dados.groupby('Date').max()
        consolidado=consolidado.transpose().round()
        #Convertendo valores para numérico
        consolidado=consolidado.apply(pd.to_numeric)
        consolidado.fillna(value=0,inplace=True)
        #Encontrando onde aconteceram as 5 maiores harmonicas do dia
        consolidado=consolidado.sort_values(by=[0],na_position='last',ascending=False).head(5)
        consolidado=consolidado.transpose()
        save.append(consolidado)
    if len(caminhos)>1:
        
        #Concatenando data frames
        Relatorio=pd.concat(save)
        #Formatando figura e arredondando valores
        Relatorio=Relatorio.round(2).astype(np.str0).style.set_caption('Maiores harmônicas em cada dia').set_table_styles([{
        'selector': 'caption',
        'props': [
        ('color', 'black'),
        ('font-size', '15px'),
        ('font-style', 'italic'),
        ('font-weight', 'bold'),
        ('text-align', 'center')]}])
    return Relatorio

In [234]:
def Make_HTML(caminho=str):
    '''Função para exportar data frames como HTML
    \ncaminho=endereço onde você deseja salvar seu arquivo
    '''
    with open(caminho, 'w') as _file:
        _file.write(ch.to_html() 
                    + "\n\n" + th.to_html()
                    + "\n\n" + rg.to_html())

In [235]:
#Extraindo dados das planilhas CSV
Geral328=Extrair_Dados_Gerais(caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220328.csv')
Geral329=Extrair_Dados_Gerais(caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220329.csv')
Geral330=Extrair_Dados_Gerais(caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220330.csv')
Geral331=Extrair_Dados_Gerais(caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220331.csv')
Geral401=Extrair_Dados_Gerais(caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220401.csv')
Geral402=Extrair_Dados_Gerais(caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220402.csv')
Geral403=Extrair_Dados_Gerais(pular_linha=1,delimitador=None,caminho='D:/Downloads2/Dados_compilado/DataSheet_3421198001_3P4W_20220403.csv')
#Formar relatório
rg=Relatorio_Geral(Geral328,Geral329,Geral330,Geral331,Geral401,Geral402)

In [236]:
#Extrair maiores harmonicas da tensão
th=Maiores_harmonicas(
    caminhos=['D:/Downloads2/Dados_compilado/VoltageHarmonics_3421198001_20220328.csv',
            'D:/Downloads2/Dados_compilado/VoltageHarmonics_3421198001_20220329.csv',
            'D:/Downloads2/Dados_compilado/VoltageHarmonics_3421198001_20220330.csv',
            'D:/Downloads2/Dados_compilado/VoltageHarmonics_3421198001_20220331.csv',
            'D:/Downloads2/Dados_compilado/VoltageHarmonics_3421198001_20220401.csv',
            'D:/Downloads2/Dados_compilado/VoltageHarmonics_3421198001_20220402.csv',
            'D:/Downloads2/Dados_compilado/VoltageHarmonics_3421198001_20220403.csv'])

In [237]:
#Extrair maiores harmonicas da corrente
ch=Maiores_harmonicas(
  caminhos=['D:/Downloads2/Dados_compilado/CurrentHarmonics_3421198001_20220328.csv',
            'D:/Downloads2/Dados_compilado/CurrentHarmonics_3421198001_20220329.csv',
            'D:/Downloads2/Dados_compilado/CurrentHarmonics_3421198001_20220330.csv',
            'D:/Downloads2/Dados_compilado/CurrentHarmonics_3421198001_20220331.csv',
            'D:/Downloads2/Dados_compilado/CurrentHarmonics_3421198001_20220401.csv',
            'D:/Downloads2/Dados_compilado/CurrentHarmonics_3421198001_20220402.csv',
            'D:/Downloads2/Dados_compilado/CurrentHarmonics_3421198001_20220403.csv'])

In [238]:
#Exportar como HTML
Make_HTML(caminho='C:/Users/andre/OneDrive/Área de Trabalho/Relatório.html')